In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("real_estate_novosibirsk.csv")
df

,area,area_raw,item_id,type_of_house,floor,floors_in_house,rooms_number,price,district,price_order_id
0,91.0,91.0,198518500076,Кирпичный,3.0,16.0,3,5100000.0,Кировский,1
1,91.0,91.0,198518500076,Кирпичный,3.0,16.0,3,5100000.0,Кировский,2
2,91.0,91.0,198518500076,Кирпичный,3.0,16.0,3,5100000.0,Кировский,3
3,18.7,18.7,257626750244,Кирпичный,4.0,5.0,Студия,1300000.0,Ленинский,1
4,NaN,NaN,367907500565,Кирпичный,4.0,4.0,4,3500000.0,Кировский,1
...,...,...,...,...,...,...,...,...,...,...
124571,33.0,33.0,832255750867,Кирпичный,10.0,10.0,Студия,2950000.0,Заельцовский,2
124572,33.0,33.0,832255750867,Кирпичный,10.0,10.0,Студия,2950000.0,Заельцовский,3
124573,51.0,51.0,833043000375,Панельный,4.0,10.0,2,3300000.0,Октябрьский,1
124574,51.0,51.0,833043000375,Панельный,4.0,10.0,2,3300000.0,Октябрьский,2


In [21]:
df = df.drop_duplicates(subset=["item_id"], keep="last")
df = df.dropna(subset=["area"])

In [22]:
df

,area,area_raw,item_id,type_of_house,floor,floors_in_house,rooms_number,price,district,price_order_id
2,91.0,91.0,198518500076,Кирпичный,3.0,16.0,3,5100000.0,Кировский,3
3,18.7,18.7,257626750244,Кирпичный,4.0,5.0,Студия,1300000.0,Ленинский,1
6,86.0,86.0,367907500565,Кирпичный,4.0,4.0,4,3500000.0,Кировский,3
9,39.0,39.0,449357000114,NaN,8.0,19.0,NaN,2400000.0,Октябрьский,3
15,56.5,56.5,456658500306,Кирпичный,4.0,5.0,2,2025000.0,Дзержинский,3
...,...,...,...,...,...,...,...,...,...,...
124566,39.0,39.0,831037250887,Кирпичный,11.0,25.0,1,3580000.0,Дзержинский,3
124569,33.0,33.0,831536250075,Кирпичный,13.0,24.0,1,3900000.0,Центральный,3
124572,33.0,33.0,832255750867,Кирпичный,10.0,10.0,Студия,2950000.0,Заельцовский,3
124574,51.0,51.0,833043000375,Панельный,4.0,10.0,2,3300000.0,Октябрьский,2


In [38]:
def process_floor(row):
    if row["floor"] == 1:
        return "first"

    if row["floor"] == row["floors_in_house"]:
        return "last"

    return "middle"


df["price_per_meter"] = df["price"] / df["area"]
df["floor_cat"] = df.apply(process_floor, axis=1)
df

,area,area_raw,item_id,type_of_house,floor,floors_in_house,rooms_number,price,district,price_order_id,price_per_meter,floor_cat
2,91.0,91.0,198518500076,Кирпичный,3.0,16.0,3,5100000.0,Кировский,3,56043.956044,middle
3,18.7,18.7,257626750244,Кирпичный,4.0,5.0,Студия,1300000.0,Ленинский,1,69518.716578,middle
6,86.0,86.0,367907500565,Кирпичный,4.0,4.0,4,3500000.0,Кировский,3,40697.674419,last
9,39.0,39.0,449357000114,NaN,8.0,19.0,NaN,2400000.0,Октябрьский,3,61538.461538,middle
15,56.5,56.5,456658500306,Кирпичный,4.0,5.0,2,2025000.0,Дзержинский,3,35840.707965,middle
...,...,...,...,...,...,...,...,...,...,...,...,...
124566,39.0,39.0,831037250887,Кирпичный,11.0,25.0,1,3580000.0,Дзержинский,3,91794.871795,middle
124569,33.0,33.0,831536250075,Кирпичный,13.0,24.0,1,3900000.0,Центральный,3,118181.818182,middle
124572,33.0,33.0,832255750867,Кирпичный,10.0,10.0,Студия,2950000.0,Заельцовский,3,89393.939394,last
124574,51.0,51.0,833043000375,Панельный,4.0,10.0,2,3300000.0,Октябрьский,2,64705.882353,middle


In [41]:
q1 = df["price_per_meter"].quantile(0.25)
q3 = df["price_per_meter"].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
q1, q3, iqr, lower_bound, upper_bound

(54739.70281543274,
 78029.5069247211,
 23289.804109288365,
 19804.996651500194,
 112964.21308865366)

In [42]:
filtered_df = df[
    (df["price_per_meter"] >= lower_bound) & (df["price_per_meter"] <= upper_bound)
]

In [43]:
df["price_per_meter"].mean(), filtered_df["price_per_meter"].mean()

(68276.58884621307, 66017.04545163628)

In [83]:
district_type_floor_agg = filtered_df.groupby(
    ["district", "type_of_house", "floor_cat"]
)["price_per_meter"].mean()

In [84]:
district_type_agg = filtered_df.groupby(["district", "type_of_house"])[
    "price_per_meter"
].mean()

In [137]:
mean_price_by_district_type = df.groupby(["district", "type_of_house"])[
    "price_per_meter"
].mean()

In [141]:
mean_price = df["price_per_meter"].mean()

In [142]:
mean_price

68276.58884621307

In [143]:
def predict(row):
    area = row["area"]
    district = row["district"]
    type_of_house = row["type_of_house"]
    return mean_price_by_district_type[district][type_of_house] * area

In [144]:
test_df = pd.read_csv("real_estate_novosibirsk.csv")

In [130]:
test_df["floor_cat"] = test_df.apply(process_floor, axis=1)

In [145]:
test_df = test_df.drop_duplicates(subset=["item_id"], keep="last")
test_df = test_df.dropna(subset=["area"])

In [146]:
test_df = test_df[test_df["price"] > 0]

In [133]:
test_df["price_per_meter"] = test_df["price"] / test_df["area"]

In [147]:
prediction = test_df.apply(predict, axis=1)

KeyError: nan

In [135]:
((test_df["price"] - prediction) / test_df["price"]).abs().mean()

0.49953227221044405

In [148]:
prediction.head()

2     5.461861e+06
3     1.209856e+06
6     5.161759e+06
9     2.747302e+06
15    3.926648e+06
dtype: float64

In [149]:
test_df["price"]

2         5100000.0
3         1300000.0
6         3500000.0
9         2400000.0
15        2025000.0
            ...    
124566    3580000.0
124569    3900000.0
124572    2950000.0
124574    3300000.0
124575    2800000.0
Name: price, Length: 39365, dtype: float64